In [6]:
import numpy as np
import pandas as pd

import scipy.stats as ss

import seaborn as sns
import matplotlib.pyplot as plt
import pingouin as pg

In [7]:
data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-daniil-kljuvak-pcp-fbf69/5lesprodyktanal/dating_data.csv')


/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [8]:
data.head()

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1


In [9]:
data.nunique()

user_id_1    1000
user_id_2    1000
group           2
is_match        2
dtype: int64

In [10]:
data.dtypes

user_id_1    int64
user_id_2    int64
group        int64
is_match     int64
dtype: object

In [11]:
data.group.value_counts()

1    9722
0    4792
Name: group, dtype: int64

In [23]:
data.head()

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1


In [5]:
#0 группа - контрольная группа (пользовались приложением со старым алгоритмом);
# 1 - экспертиментальная группа (пользовались новым алгоритмом для поиска анкет).

In [12]:
# Группируем данные по пользователям и группам, чтобы подсчитать действия и совпадения
aggregate = (
    data.groupby(['user_id_1', 'group'], as_index=False)
      .agg({
          'user_id_2': 'count',  # Подсчитываем количество действий пользователя (лайков/дизлайков)
          'is_match': 'sum'      # Суммируем количество взаимных лайков (совпадений)
      })
      .rename(columns={
          'user_id_1': 'user_id',
          'user_id_2': 'actions',
          'is_match': 'matchs_count'
      })
)

# Рассчитываем долю совпадений среди действий пользователя
aggregate['matchs_share'] = round(aggregate['matchs_count'] / aggregate['actions'], 2)

# Выводим результирующий DataFrame
aggregate


,user_id,group,actions,matchs_count,matchs_share
0,1,1,24,11,0.46
1,2,1,16,7,0.44
2,3,1,16,5,0.31
3,4,0,9,2,0.22
4,5,1,22,13,0.59
...,...,...,...,...,...
995,996,0,8,1,0.12
996,997,0,12,1,0.08
997,998,1,18,10,0.56
998,999,0,7,2,0.29


In [13]:
aggregate.group.value_counts()

0    501
1    499
Name: group, dtype: int64

In [14]:
aggregate.query('group == 0').matchs_share.describe()

count    501.000000
mean       0.193593
std        0.134368
min        0.000000
25%        0.100000
50%        0.180000
75%        0.270000
max        0.670000
Name: matchs_share, dtype: float64

In [15]:
aggregate.query('group == 1').matchs_share.describe()

count    499.000000
mean       0.402405
std        0.114209
min        0.000000
25%        0.330000
50%        0.400000
75%        0.480000
max        0.720000
Name: matchs_share, dtype: float64

In [16]:
aggregate.query('group == 0').matchs_share

3      0.22
9      0.50
11     0.08
13     0.00
14     0.15
       ... 
993    0.22
994    0.25
995    0.12
996    0.08
998    0.29
Name: matchs_share, Length: 501, dtype: float64

In [17]:
aggregate.query('group == 1').matchs_share

0      0.46
1      0.44
2      0.31
4      0.59
5      0.41
       ... 
986    0.35
991    0.38
992    0.33
997    0.56
999    0.52
Name: matchs_share, Length: 499, dtype: float64

In [18]:
control = aggregate.query("group == 0").matchs_share
test = aggregate.query("group == 1").matchs_share

In [19]:
pg.ttest(control, test)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-26.481432,973.937119,two-sided,7.890669e-117,"[-0.22, -0.19]",1.674296,3.052e+113,1.0


In [20]:
ss.ttest_ind(control, test, equal_var = False)

Ttest_indResult(statistic=-26.481431782585016, pvalue=7.890669157069948e-117)

In [21]:
aggregate.groupby("group").mean()

,user_id,actions,matchs_count,matchs_share
group,,,,
0,497.371257,9.564870,1.864271,0.193593
1,503.641283,19.482966,7.833667,0.402405


In [31]:
control_actions = aggregate.query('group == 0').actions
test_actions = aggregate.query('group == 1').actions

In [32]:
ss.ttest_ind(control_actions, test_actions, equal_var=True)

Ttest_indResult(statistic=-51.85383774946492, pvalue=1.8942877064043142e-285)